In [1]:
import sys
sys.path.append('..')

import torch
import transformers
import peft
import accelerate
import datasets
import copy
import wandb

In [2]:
from src import settings
from utils.measure_resource import gpu_information
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import Trainer, TrainingArguments
from tqdm import tqdm
from torch.utils.data import Dataset
wandb_api_key = settings.WANDB_API_KEY

In [ ]:
gpu_information()

In [5]:
base_model_name = 'line-corporation/japanese-large-lm-3.6b-instruction-sft'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_fast=False)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map = 'auto')

In [ ]:
for param in base_model.parameters():
    # ベースモデルのパラメータは勾配計算の対象外とします。
    param.requires_grad = False
    # 学習に使用するレイヤーのパラメータの型を float32 にして精度を上げます。
    if param.ndim == 1:
        param.data = param.data.to(torch.float32)

# メモリを節約するために、Gradient Checkpointing アルゴリズムを有効化します。
base_model.gradient_checkpointing_enable()

# モデルの重みを固定したままアダプターの重みを調整するために、入力埋め込みグラデーションを有効化します。
base_model.enable_input_require_grads()

class CastOutputToFloat(torch.nn.Sequential):
   def forward(self, x):
      return super().forward(x).to(torch.float32)

base_model.embed_out = CastOutputToFloat(base_model.embed_out)

print(base_model)

In [8]:
peft_config = peft.LoraConfig(
    r=256,
    lora_alpha=32,
    target_modules=['query_key_value'],
    lora_dropout=0.05,
    bias='none',
    fan_in_fan_out=False,
    task_type=peft.TaskType.CAUSAL_LM
)

In [ ]:
peft_model = peft.get_peft_model(base_model, peft_config)
peft_model.print_trainable_parameters()

In [10]:
dataset_name = 'kunishou/databricks-dolly-15k-ja'
dataset = datasets.load_dataset(dataset_name)

In [11]:
dolly_ja = list(dataset['train'])

In [12]:
PROMPT_DICT = {
    "prompt_input": (
        "以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。"
        "要求を適切に満たす応答を書きなさい。\n\n"
        "### 指示:\n{instruction}\n\n### 入力:{input}\n\n### 応答:"
    ),
    "prompt_no_input": (
        "以下は、タスクを説明する指示です。"
        "要求を適切に満たす応答を書きなさい。\n\n"
        "### 指示:\n{instruction}\n\n### 応答:"
    )
}

In [13]:
class InstructDataset(Dataset):
    def __init__(self, json_list, tokenizer, ignore_index=-100):
        self.tokenizer = tokenizer
        self.ignore_index = ignore_index
        self.features = []
        
        for j in tqdm(json_list):
            # open_qaなど文脈情報が必要ない場合はinputカラムがないため、
            # inputカラムありなしでテンプレート文を分けている。
            if 'input' in j:
                source_text = PROMPT_DICT['prompt_input'].format_map(j)
            else:
                source_text = PROMPT_DICT['prompt_no_input'].format_map(j)
            
            # 指示文と回答文を結合し、文末にEOSトークンを挿入
            example_text = source_text + j['output'] + self.tokenizer.eos_token
            
            # 指示文のみ（「以下は、タスクを〜### 応答:」まで）をtokenize
            # ほしいのは指示文のlength
            source_tokenized = self.tokenizer(
                source_text,
                padding='longest',
                truncation=True,
                max_length=1024,
                return_length=True,
                return_tensors='pt'
            )
            
            # 指示文と回答文を全てtokenize
            example_tokenized = self.tokenizer(
                example_text, 
                padding='longest', 
                truncation=True, 
                max_length=1024, 
                return_tensors='pt'
            )
            
            input_ids = example_tokenized['input_ids'][0]
            
            # LLMが生成してほしい正解の文章として入力文をそのままコピーする
            labels = copy.deepcopy(input_ids)
            
            # 指示文までの長さ
            source_len = source_tokenized['length'][0]
            
            # LLMに生成してほしい正解文章に指示文も含まれているので、
            # 指示文のところはCrossEntropyLossの損失を計算をしないようにIGNORE_INDEXとして-100で埋める
            labels[:source_len] = self.ignore_index
            
            self.features.append({
                'input_ids': input_ids,
                'labels': labels
            })
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx]

In [14]:
class InstructCollator():
    def __init__(self, tokenizer, ignore_index=-100):
        self.tokenizer = tokenizer
        self.ignore_index = -100

    def __call__(self, examples):
        input_batch = []
        label_batch = []
        for example in examples:
            input_batch.append(example['input_ids'])
            label_batch.append(example['labels'])
        
        input_ids = pad_sequence(
            input_batch, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )

        # labelsのpaddingトークンは先程と同様にignore_indexである-100で埋める
        labels = pad_sequence(
            label_batch, batch_first=True, padding_value=self.ignore_index
        )

        # attention_maskはbool値でもいいらしい
        attention_mask = input_ids.ne(self.tokenizer.pad_token_id)
            
        return {
            'input_ids': input_ids,
            'labels': labels,
            'attention_mask': attention_mask
        }

In [ ]:
dataset =InstructDataset(dolly_ja, tokenizer)
collator = InstructCollator(tokenizer)

In [ ]:
wandb.login(
    key = wandb_api_key
)

In [23]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    warmup_steps=100,
    learning_rate=2e-4,
    fp16=True,
    num_train_epochs=2,
    save_strategy="steps",
    save_steps=100,
    output_dir=".checkpoints",
    eval_strategy="no",
    logging_dir="logs",
    logging_steps=10,
    push_to_hub=False,
    report_to="wandb"
)

In [24]:
trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=dataset,
    args=training_args,
    data_collator=collator,
)

In [25]:
peft_model.config.use_cache = False

In [26]:
checkpoint = None

In [ ]:
trainer.train(checkpoint)
peft_model.config.use_cache = True

In [ ]:
peft_model.save_pretrained("../result/japanese-large-lm-3.6b-instruction-sft-r256/")